In [ ]:
import re
import regex
import pandas as pd
from function import revers,replace_ambiguity_codes

def revers(seq):
    codes={"A":"A", "C":"C", "G":"G", "T":"T", 
           "R":"[AG]", "S":"[GC]", "B":"[CGT]", "Y":"[CT]", 
           "W":"[AT]", "D":"[AGT]", "K":"[GT]", "N":"[ACGT]", 
           "H":"[ACT]", "M":"[AC]", "V":"[ACG]", "X": "[ACGT]"}
    
    d={'A':'T','G':'C','C':'G','T':'A',
      'R':'Y','S':'S','B':'V','Y':'R',
       'W':'W','D':'H','K':'M','N':'N',
       'H':'D','M':'K','V':'B','X':'X'
      }
    new=''
    for i in seq[::-1]:
        new+=d[i]
    return new

def replace_ambiguity_codes(sequence):
    codes={"A":"A", "C":"C", "G":"G", "T":"T", 
           "R":"[AG]", "S":"[GC]","B":"[CGT]", "Y":"[CT]", "W":"[AT]",
           "D":"[AGT]", "K":"[GT]", "N":"[ACGT]", "H":"[ACT]", "M":"[AC]", "V":"[ACG]", "X": "[ACGT]"}
    regexlist=[]
    for base in sequence:
        if not base in codes:
            print("Unidentified nucleotide code in primer:", base)
            sys.exit()
        else:
            regexlist.append(codes[base])

    return ''.join(regexlist)

def getposition(match,matches):
    
    a = re.finditer(match, matches)
    for match in a:
        start_pos = match.start()
        end_pos = match.end()

        # Get the matched text itself
        matched_text = match.group()

        # Print the match and its positions
        aa=f"Match: '{matched_text}' | Start: {start_pos} | End: {end_pos}"
        return aa
def coverage(forward_list, reverse_list, d):
    coverage ={}
    for each_class in d:
        c=0
        for i in d[each_class]:
            f=0
            r=0
            pcr_product=[]
            a=[];b=[]
            for forward_primer in forward_list: 
                f_b = re.findall(forward_primer,i)
                if len(f_b) == 1:
                    a.append(getposition(f_b[0],i))
            for reverse_primer in reverse_list:
                r_b = re.findall(revers(reverse_primer),i)
                if len(r_b) == 1:
                    b.append(getposition(r_b[0],i))
            if len(a)>0 and len(b)>0:
                c+=1
        coverage[each_class]=format(c*100/len(d[each_class]),'.1f')+'%'
    return coverage
def coverage_e1(forward_list, reverse_list, d):
    coverage ={}
    for each_class in d:
        c=0
        for i in d[each_class]:
            f=0
            r=0
            pcr_product=[]
            a=[];b=[]
            for forward_primer in forward_list: 
                f_b = regex.findall("("+forward_primer+"){e<=1}",i)
                if len(f_b) == 1:
                    a.append(getposition(f_b[0],i))
            for reverse_primer in reverse_list:
                r_b = regex.findall("("+revers(reverse_primer)+"){e<=1}",i)
                if len(r_b) == 1:
                    b.append(getposition(r_b[0],i))
            if len(a)>0 and len(b)>0:
                c+=1
        coverage[each_class]=format(c*100/len(d[each_class]),'.1f')+'%'
    return coverage

def cov(forward_primer, reverse_primer, d):
    coverage ={}
    for each_class in d:
        c=0
        for i in d[each_class]:
            f=0
            r=0
            pcr_product=[]
            a=[];b=[]
            f_b = re.findall(replace_ambiguity_codes(forward_primer),i)
            if len(f_b) == 1:
                a.append(getposition(f_b[0],i))
            r_b = re.findall(replace_ambiguity_codes(revers(reverse_primer)),i)
            if len(r_b) == 1:
                b.append(getposition(r_b[0],i))
            if len(a)>0 and len(b)>0:
                c+=1
        coverage[each_class]=format(c*100/len(d[each_class]),'.1f')+'%'
    return coverage
def cov_e1(forward_primer, reverse_primer, d):
    coverage ={}
    for each_class in d:
        c=0
        for i in d[each_class]:
            f=0
            r=0
            pcr_product=[]
            a=[];b=[]
            if 2>1:
                f_b = regex.findall("("+replace_ambiguity_codes(forward_primer)+"){e<=1}",i)
                if len(f_b) == 1:
                    a.append(getposition(f_b[0],i))
            if 2>1:
                r_b = regex.findall("("+replace_ambiguity_codes(revers(reverse_primer))+"){e<=1}",i)
                if len(r_b) == 1:
                    b.append(getposition(r_b[0],i))
            if len(a)>0 and len(b)>0:
                c+=1
        coverage[each_class]=format(c*100/len(d[each_class]),'.1f')+'%'
    return coverage

# SILVA r138.1

In [ ]:
# p: SIlVA phylum level dict
# c: SILVA Class level dict

p={};c={}
top20=["Bacteroidota",
       "Proteobacteria",
       "Actinobacteriota",
       "Chloroflexi",
       "Verrucomicrobiota",
       "Myxococcota",
       "Firmicutes",
       "Planctomycetota",
       "Acidobacteriota",
       "Campylobacterota",
       "Nitrospirota",
       "Patescibacteria",
       "Desulfobacterota",
       "Bdellovibrionota",
       "Cyanobacteria",
       "Spirochaetota",
       "Fusobacteriota",
       "Euryarchaeota",
       "Latescibacterota",
       "Armatimonadota"]
for i in top20:
    p[i]=[]
p['Others'] = []
#https://www.arb-silva.de/no_cache/download/archive/release_138.1/Exports/
with open(r'SILVA_138.1_SSURef_NR99_tax_silva.fasta') as f:
    data=('\n'+f.read()).split('\n>')[1:]
    for i in data:
        if 'Patescibacteria' in i:
            if i.split('\n')[0].split(';')[2] not in c:
                c[i.split('\n')[0].split(';')[2]] = [i.split('\n',1)[1].replace('U','T').replace('\n','')]
            else:
                c[i.split('\n')[0].split(';')[2]].append(i.split('\n',1)[1].replace('U','T').replace('\n',''))
        if 'Eukaryota' not in i:
            if i.split('\n')[0].split(';')[1] not in top20:
                p['Others'].append(i.split('\n',1)[1].replace('U','T').replace('\n',''))
            else:
                p[i.split('\n')[0].split(';')[1]].append(i.split('\n',1)[1].replace('U','T').replace('\n',''))
c2 = {}
c2['Others'] = []
common_class=['Parcubacteria', 'Gracilibacteria', 'Microgenomatia', 'Berkelbacteria', 'ABY1', 'Saccharimonadia', 'Dojkabacteria', 'WWE3']
for i in c:
    if i in common_class:
        c2[i] = c[i]
    else:
        for j in c[i]:
            c2['Others'].append(j)

# MiDAS 4.8.1

In [ ]:
# midas : MIDAS phylum dict contain seqs
# mc(2): MiDAS Class level dict contain seqs
midas={}
mc = {}
top20=["Bacteroidota",
       "Proteobacteria",
       "Actinobacteriota",
       "Chloroflexi",
       "Verrucomicrobiota",
       "Myxococcota",
       "Firmicutes",
       "Planctomycetota",
       "Acidobacteriota",
       "Campylobacterota",
       "Nitrospirota",
       "Patescibacteria",
       "Desulfobacterota",
       "Bdellovibrionota",
       "Cyanobacteria",
       "Spirochaetota",
       "Fusobacteriota",
       "Euryarchaeota",
       "Latescibacterota",
       "Armatimonadota"]
for i in top20:
    midas[i]=[]
midas['Others'] = []
# can be accessed here: https://www.midasfieldguide.org/guide/downloads
with open(r'SINTAX fa file MiDAS 4.8.1.fa') as f:
    data=('\n'+f.read().replace("tax=",'')).split('\n>')[1:]
    for i in data:
        if i.split(',')[1].split('\n')[0].replace('p:','') in top20:
            midas[i.split(',')[1].split('\n')[0].replace('p:','')].append(i.split('\n',1)[1].replace('\n',''))
        else:
            midas['Others'].append(i.split('\n',1)[1].replace('\n',''))
        if 'Patesci' in i:
            if i.split(',')[2].replace('c:','') not in mc:
                mc[i.split(',')[2].replace('c:','')] = [i.split('\n',1)[1].replace('\n','')]
            else:
                mc[i.split(',')[2].replace('c:','')].append(i.split('\n',1)[1].replace('\n',''))
mc2= {}
mc2['Others'] = []
common_class=['Parcubacteria', 'Gracilibacteria', 'Microgenomatia', 'Berkelbacteria', 'ABY1', 'Saccharimonadia', 'Dojkabacteria', 'WWE3']

for i in mc:
    if i in common_class:
        mc2[i] = mc[i]
    else:
        for j in mc[i]:
            mc2['Others'].append(j)


# primer sequences used in analysis

In [ ]:
V4_515Yp = '''GTGCCAGCAGCAGCGGTAA
GTGCCAGCACCCGCGGTAA
GTGCCAGCAGACGCGGTAA
GTGCCAGCAGCCGCGGTAA
GTGCCAGCAGCCGCGGTCA
GTGCCAGCAGCCGCGGTGA
GTGCCAGCAGCTGCGGTAA
GTGCCAGCAGTCGCGGCAA
GTGCCAGCAGTCGCGGTAA
GTGCCAGCCGCCGCGGTAA
GTGGCAGCCGCCGCGGTAA
GTGTCAGCAGCCGCGGTAA
GTGTCAGCCGCCGCGGTAA'''.split('\n')
V4_926Rp = '''CCGTCAATTCGTTTAAGTTT
CCACCAGAACATTTAAGTTT
CCATCAATTCATTTGAGTTT
CCGCCAATTCATTTAAGTTT
CCGCCAATTCATTTGAGTTT
CCGCCAATTCCCTTAAGTCT
CCGCCAATTCCCTTAAGTTT
CCGCCAATTCCTATAAGTTT
CCGCCAATTCCTTCAAGTTT
CCGCCAATTCCTTTAAGTCG
CCGCCAATTCCTTTAAGTCT
CCGCCAATTCCTTTAAGTTA
CCGCCAATTCCTTTAAGTTT
CCGCCAATTCCTTTGAGTTT
CCGCCAATTGGTTTAAGTTT
CCGCCAATTTCTTTAAGTTT
CCGCCAATTTCTTTGAGTTT
CCGTCAACTCATTTGAGTTT
CCGTCAATCCCTTTAAGTTT
CCGTCAATGCCTTTAAGTTT
CCGTCAATTACTTTAAGTTT
CCGTCAATTACTTTGAGTTT
CCGTCAATTCATTTAAGTTT
CCGTCAATTCATTTGAGTTT
CCGTCAATTCCCTTGAGTTT
CCGTCAATTCCTTCAAGTTT
CCGTCAATTCCTTTAAATTT
CCGTCAATTCCTTTAAGTTT
CCGTCAATTCCTTTGAATTT
CCGTCAATTCCTTTGAGTTC
CCGTCAATTCCTTTGAGTTT
CCGTCAATTCCTTTGGGTTC
CCGTCAATTCCTTTGGGTTT
CCGTCAATTCTTTTGAGTTT
CCGTCAATTTATTTGAGTTT
CCGTCAATTTCTTTAAGTTT
CCGTCAATTTCTTTGAGTTT
CCGTCTATTCCTTTGAGTTT'''.split('\n')

# Coverage calculate

In [ ]:
# _0 : 0 mismatch
# _1: 1 mismatch
silva_class_0 = coverage(V4_515Yp,V4_926Rp,c2)
silva_phylum_0 = coverage(V4_515Yp,V4_926Rp,p)
silva_class_1 = coverage_e1(V4_515Yp,V4_926Rp,c2)
silva_phylum_1 = coverage_e1(V4_515Yp,V4_926Rp,p)
midas_class_0 = coverage(V4_515Yp,V4_926Rp,mc2)
midas_phylum_0 = coverage(V4_515Yp,V4_926Rp,midas)
midas_class_1 = coverage_e1(V4_515Yp,V4_926Rp,mc2)
midas_phylum_1 = coverage_e1(V4_515Yp,V4_926Rp,midas)

v13_p_0=cov('AGAGTTTGATCCTGGCTCAG','ATTACCGCGGCTGCTGG',p) #27F 534R 
v13_p_1=cov_e1('AGAGTTTGATCCTGGCTCAG','ATTACCGCGGCTGCTGG',p) #27F 534R 
v13_midas_0=cov('AGAGTTTGATCCTGGCTCAG','ATTACCGCGGCTGCTGG',midas) #27F 534R 
v13_midas_1=cov_e1('AGAGTTTGATCCTGGCTCAG','ATTACCGCGGCTGCTGG',midas) #27F 534R 

v4_p_0=cov('GTGYCAGCMGCCGCGGTAA','GGACTACNVGGGTWTCTAAT',p) #515F (Parada)–806R (Apprill)
v4_p_1=cov_e1('GTGYCAGCMGCCGCGGTAA','GGACTACNVGGGTWTCTAAT',p) #515F (Parada)–806R (Apprill)
v4_midas_0=cov('GTGYCAGCMGCCGCGGTAA','GGACTACNVGGGTWTCTAAT',midas) #515F (Parada)–806R (Apprill)
v4_midas_1=cov_e1('GTGYCAGCMGCCGCGGTAA','GGACTACNVGGGTWTCTAAT',midas) #515F (Parada)–806R (Apprill)

v34_p_0=cov('CCTACGGGNGGCWGCAG','GACTACHVGGGTATCTAATCC',p) #341F 806R
v34_p_1=cov_e1('CCTACGGGNGGCWGCAG','GACTACHVGGGTATCTAATCC',p) #341F 806R
v34_midas_0=cov('CCTACGGGNGGCWGCAG','GACTACHVGGGTATCTAATCC',midas) #341F 806R
v34_midas_1=cov_e1('CCTACGGGNGGCWGCAG','GACTACHVGGGTATCTAATCC',midas) #341F 806R

v4_cpr_p_0=cov("GTGYCAGMAGBNKCGGTVA","RGACTAMNVRGGTHTCTAAT",p)
v4_cpr_p_1=cov_e1("GTGYCAGMAGBNKCGGTVA","RGACTAMNVRGGTHTCTAAT",p)
v4_cpr_midas_0=cov("GTGYCAGMAGBNKCGGTVA","RGACTAMNVRGGTHTCTAAT",midas)
v4_cpr_midas_1=cov_e1("GTGYCAGMAGBNKCGGTVA","RGACTAMNVRGGTHTCTAAT",midas)


v13_c2_0=cov('AGAGTTTGATCCTGGCTCAG','ATTACCGCGGCTGCTGG',c2) #27F 534R 
v13_c2_1=cov_e1('AGAGTTTGATCCTGGCTCAG','ATTACCGCGGCTGCTGG',c2) #27F 534R 
v13_mc2_0=cov('AGAGTTTGATCCTGGCTCAG','ATTACCGCGGCTGCTGG',mc2) #27F 534R 
v13_mc2_1=cov_e1('AGAGTTTGATCCTGGCTCAG','ATTACCGCGGCTGCTGG',mc2) #27F 534R 

v4_c2_0=cov('GTGYCAGCMGCCGCGGTAA','GGACTACNVGGGTWTCTAAT',c2) #515F (c2arada)–806R (Ac2c2rill)
v4_c2_1=cov_e1('GTGYCAGCMGCCGCGGTAA','GGACTACNVGGGTWTCTAAT',c2) #515F (c2arada)–806R (Ac2c2rill)
v4_mc2_0=cov('GTGYCAGCMGCCGCGGTAA','GGACTACNVGGGTWTCTAAT',mc2) #515F (c2arada)–806R (Ac2c2rill)
v4_mc2_1=cov_e1('GTGYCAGCMGCCGCGGTAA','GGACTACNVGGGTWTCTAAT',mc2) #515F (c2arada)–806R (Ac2c2rill)

v34_c2_0=cov('CCTACGGGNGGCWGCAG','GACTACHVGGGTATCTAATCC',c2) #341F 806R
v34_c2_1=cov_e1('CCTACGGGNGGCWGCAG','GACTACHVGGGTATCTAATCC',c2) #341F 806R
v34_mc2_0=cov('CCTACGGGNGGCWGCAG','GACTACHVGGGTATCTAATCC',mc2) #341F 806R
v34_mc2_1=cov_e1('CCTACGGGNGGCWGCAG','GACTACHVGGGTATCTAATCC',mc2) #341F 806R

v4_cc2r_c2_0=cov("GTGYCAGMAGBNKCGGTVA","RGACTAMNVRGGTHTCTAAT",c2)
v4_cc2r_c2_1=cov_e1("GTGYCAGMAGBNKCGGTVA","RGACTAMNVRGGTHTCTAAT",c2)
v4_cc2r_mc2_0=cov("GTGYCAGMAGBNKCGGTVA","RGACTAMNVRGGTHTCTAAT",mc2)
v4_cc2r_mc2_1=cov_e1("GTGYCAGMAGBNKCGGTVA","RGACTAMNVRGGTHTCTAAT",mc2)   

# Write result

In [ ]:
with open(r'All_Silva_phylum.tsv','w') as of:
    of.write('Class\tSequences number\tV1-V3_0\tV1-V3_e1\tV3-V4_0\tV3-V4_e1\tV4-V5\tV4-V5_e1\tV4_0\tV4_e1\tV4_Mod\tV4_Mod_e1\n')
    for i in p:
        of.write(i+'\t'+str(len(p[i]))+'\t' \
                 +v13_p_0[i]+'\t'+v13_p_1[i]+'\t'\
                 +v34_p_0[i]+'\t'+v34_p_1[i]+'\t'\
                 +silva_phylum_0[i]+'\t'+silva_phylum_1[i]+'\t'\
                 +v4_p_0[i]+'\t'+v4_p_1[i]+'\t'\
                 +v4_cpr_p_0[i]+'\t'+v4_cpr_p_1[i]+'\n')
with open(r'All_Silva_class.tsv','w') as of:
    of.write('Class\tSequences number\tV1-V3_0\tV1-V3_e1\tV3-V4_0\tV3-V4_e1\tV4-V5\tV4-V5_e1\tV4_0\tV4_e1\tV4_Mod\tV4_Mod_e1\n')
    for i in c2:
        of.write(i+'\t'+str(len(c2[i]))+'\t' \
                 +v13_c2_0[i]+'\t'+v13_c2_1[i]+'\t'\
                 +v34_c2_0[i]+'\t'+v34_c2_1[i]+'\t'\
                 +silva_class_0[i]+'\t'+silva_class_1[i]+'\t'\
                 +v4_c2_0[i]+'\t'+v4_c2_1[i]+'\t'\
                 +v4_cc2r_c2_0[i]+'\t'+v4_cc2r_c2_1[i]+'\n')
with open(r'All_midas_phylum.tsv','w') as of:
    of.write('Class\tSequences number\tV1-V3_0\tV1-V3_e1\tV3-V4_0\tV3-V4_e1\tV4-V5\tV4-V5_e1\tV4_0\tV4_e1\tV4_Mod\tV4_Mod_e1\n')
    for i in midas:
        of.write(i+'\t'+str(len(midas[i]))+'\t' \
                 +v13_midas_0[i]+'\t'+v13_midas_1[i]+'\t'\
                 +v34_midas_0[i]+'\t'+v34_midas_1[i]+'\t'\
                 +midas_phylum_0[i]+'\t'+midas_phylum_1[i]+'\t'\
                 +v4_midas_0[i]+'\t'+v4_midas_1[i]+'\t'\
                 +v4_cpr_midas_0[i]+'\t'+v4_cpr_midas_1[i]+'\n')
with open(r'All_midas_class.tsv','w') as of:
    of.write('Class\tSequences number\tV1-V3_0\tV1-V3_e1\tV3-V4_0\tV3-V4_e1\tV4-V5\tV4-V5_e1\tV4_0\tV4_e1\tV4_Mod\tV4_Mod_e1\n')
    for i in mc2:
        of.write(i+'\t'+str(len(mc2[i]))+'\t' \
                 +v13_mc2_0[i]+'\t'+v13_mc2_1[i]+'\t'\
                 +v34_mc2_0[i]+'\t'+v34_mc2_1[i]+'\t'\
                 +midas_class_0[i]+'\t'+midas_class_1[i]+'\t'\
                 +v4_mc2_0[i]+'\t'+v4_mc2_1[i]+'\t'\
                 +v4_cc2r_mc2_0[i]+'\t'+v4_cc2r_mc2_1[i]+'\n')
    